In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import csv

In [2]:
# File path to your CSV file
file_path = r"C:\Users\jdimi\OneDrive\Desktop\Data Analyist Bootcamp\Project 4\project-4\Resource\inspection_dataset.csv"

# Load the CSV into a DataFrame
df = pd.read_csv(file_path)
df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_score,inspection_type,violation_id,violation_description,risk_category,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,85936,Laurel Court,950 Mason St,San Francisco,CA,94108,NaN,NaN,NaN,1.415578e+10,...,100.0,Routine - Unscheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5827,HILLCREST ELEMENTARY SCHOOL,810 SILVER Ave,San Francisco,CA,94134,37.729016,-122.419253,POINT (-122.419253 37.729016),1.415546e+10,...,NaN,Reinspection/Followup,NaN,NaN,NaN,92.0,92.0,2.0,2.0,7.0
2,94910,Ike's Kitchen,800 Van Ness Ave,San Francisco,CA,94109,NaN,NaN,NaN,NaN,...,NaN,New Ownership - Followup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64667,Jasmine Rae Bakery,1890 Bryant St #309,San Francisco,CA,94110,37.763156,-122.410351,POINT (-122.410351 37.763156),NaN,...,NaN,Reinspection/Followup,NaN,NaN,NaN,53.0,53.0,3.0,2.0,20.0
4,97722,THE CHURRO FACTORY,PIER 39 K-01,San Francisco,CA,94133,NaN,NaN,NaN,NaN,...,96.0,Routine - Unscheduled,97722_20181217_103154,Unclean or degraded floors walls or ceilings,Low Risk,NaN,NaN,NaN,NaN,NaN


In [3]:
# Count total numbr of rows
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53973 entries, 0 to 53972
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   business_id                   53973 non-null  int64  
 1   business_name                 53973 non-null  object 
 2   business_address              53973 non-null  object 
 3   business_city                 53973 non-null  object 
 4   business_state                53973 non-null  object 
 5   business_postal_code          52787 non-null  object 
 6   business_latitude             27475 non-null  float64
 7   business_longitude            27475 non-null  float64
 8   business_location             27460 non-null  object 
 9   business_phone_number         17672 non-null  float64
 10  inspection_id                 53973 non-null  object 
 11  inspection_date               53973 non-null  object 
 12  inspection_score              39541 non-null  float64
 13  i

In [4]:
# Total number of unique business in the data
total_business = df['business_name'].nunique()
print(total_business)

5775


In [5]:
# Total number of unique zip codes
total_zip_codes = df['business_postal_code'].nunique()
print(total_zip_codes)

62


In [6]:
# Total number of rows for each inspection type
inspection_type_counts = df['inspection_type'].value_counts()

print(inspection_type_counts)

inspection_type
Routine - Unscheduled                39638
Reinspection/Followup                 6718
Complaint                             2399
New Ownership                         1914
New Construction                       999
Non-inspection site visit              847
New Ownership - Followup               531
Structural Inspection                  402
Complaint Reinspection/Followup        232
Foodborne Illness Investigation        203
Routine - Scheduled                     74
Special Event                            8
Administrative or Document Review        4
Multi-agency Investigation               3
Community Health Assessment              1
Name: count, dtype: int64


In [7]:
# Total number of unique inspections
total_inspections = df['inspection_id'].nunique()
print(total_inspections)

26663


In [8]:
# Total number of unique inspections per inspection type
total_inspections_per_type = df.groupby('inspection_type')['inspection_id'].nunique()
print(total_inspections_per_type.sort_values(ascending=False))

inspection_type
Routine - Unscheduled                14077
Reinspection/Followup                 6439
New Ownership                         1592
Complaint                             1458
New Construction                       994
Non-inspection site visit              811
New Ownership - Followup               499
Structural Inspection                  394
Complaint Reinspection/Followup        227
Foodborne Illness Investigation        115
Routine - Scheduled                     46
Administrative or Document Review        4
Multi-agency Investigation               3
Special Event                            3
Community Health Assessment              1
Name: inspection_id, dtype: int64


In [9]:
# Define the base URL of the website
base_url = "https://inspections.myhealthdepartment.com"

# Specify the path to the ChromeDriver executable
driver_path = r"C:/Users/jdimi/OneDrive/Desktop/Chrome Driver/chromedriver-win64/chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the initial page to extract all listings
start_url = f"{base_url}/san-francisco"
driver.get(start_url)

# Wait for the page to load
time.sleep(10) 

In [10]:
# List to hold scraped data
scraped_data = []

# Find and scrape businesses
while True:
    try:
        # Find all businesses in the list
        businesses = driver.find_elements(By.CLASS_NAME, 'flex-row')
        for index, business in enumerate(businesses):
            try:
                # Extract main business details
                business_name = business.find_element(By.CLASS_NAME, 'establishment-list-name').text.strip()
                address = business.find_element(By.CLASS_NAME, 'establishment-list-address').text.strip()
                inspection_type = business.find_element(By.CLASS_NAME, 'text-right').text.strip().split('\n')[0]
                inspection_date = business.find_element(By.CLASS_NAME, 'text-right').text.strip().split('\n')[1]

                # Save link to detail page
                view_inspection_link = business.find_element(By.TAG_NAME, 'a').get_attribute('href')

                # Navigate to detail page
                driver.get(view_inspection_link)

                # Wait for the detail page to load
                time.sleep(2)

                # Scrape additional details if needed (e.g., inspection purpose)
                # Add placeholders if not scraping extra details
                inspection_purpose = 'N/A'

                # Append scraped data
                scraped_data.append({
                    "business_name": business_name,
                    "business_address": address,
                    "inspection_type": inspection_type,
                    "inspection_date": inspection_date,
                    "inspection_purpose": inspection_purpose,
                })

                # Navigate back to the main page
                driver.back()
                time.sleep(2)

                # Refresh businesses list after navigating back
                businesses = driver.find_elements(By.CLASS_NAME, 'flex-row')

            except Exception as e:
                print(f"Error scraping data for a business: {e}")
                continue

        # Break out of the loop after scraping all businesses
        break

    except Exception as e:
        print(f"Unexpected error: {e}")
        break

# Quit the application
driver.quit()

In [11]:
# Save scraped data to a DataFrame and CSV
scraped_df = pd.DataFrame(scraped_data)
scraped_df.to_csv('detailed_inspection_data.csv', index=False)

print("Scraping completed and saved to 'detailed_inspection_data.csv'!")

Scraping completed and saved to 'detailed_inspection_data.csv'!


In [12]:
# Merge the new data with the base CSV
merged_df = pd.merge(df, scraped_df, how="left", on="business_name")

# Save the merged data to a new CSV
merged_csv_path = "merged_data.csv"
merged_df.to_csv(merged_csv_path, index=False)

print("Scraping and merging complete!")

Scraping and merging complete!
